In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

# 페이지 정보를 가져오는 함수
def get_page_info(contestno):
    url = f'https://gongmo.incruit.com/info/gongmolistinfo.asp?contestno={contestno}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    # HTTP 요청을 보내고 페이지 내용을 받음
    response = requests.get(url, headers=headers)
    
    # 요청이 성공했는지 확인
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # 필요한 데이터 추출
        category = soup.find('p', class_='view--category').get_text(strip=True) if soup.find('p', class_='view--category') else '정보 없음'
        title = soup.find('h1', class_='gongmo-view-contents__h1').get_text(strip=True) if soup.find('h1', class_='gongmo-view-contents__h1') else None
        
        # 제목이 없으면 데이터 저장하지 않음
        if not title:
            return None
        
        # 주최자 정보 추출 (p 태그가 포함된 td를 찾아서 텍스트 추출)
        organizer_td = soup.find_all('td')[0]  # find_all로 모든 td 요소를 찾음
        if organizer_td:
            organizer = organizer_td.find('p').get_text(strip=True) if organizer_td.find('p') else '정보 없음'
        else:
            organizer = '정보 없음'

        # 진행상태 정보 추출 (None 체크 추가)
        status_span = soup.find_all('td')[1].get_text(strip=True) if len(soup.find_all('td')) > 1 else '정보 없음'
        # 접수기간 정보 추출
        period_td = soup.find_all('td')[2].get_text(strip=True) if len(soup.find_all('td')) > 2 else '정보 없음'

        # 응모대상 정보 추출
        target = soup.find_all('td')[3].get_text(strip=True) if len(soup.find_all('td')) > 3 else '정보 없음'

        # 시상내역 정보 추출
        prize = soup.find_all('td')[4].get_text(strip=True) if len(soup.find_all('td')) > 4 else '정보 없음'

        # 링크 정보 추출
        link = soup.find('a', class_='c-btn c-btn-orange c-btn-xlg')['href'] if soup.find('a', class_='c-btn c-btn-orange c-btn-xlg') else '정보 없음'

        # 이미지 정보 추출
        image_div = soup.find('div', class_='gongmo-aside-right__thumbnail')
        if image_div and image_div.find('img'):
            image = image_div.find('img')['src']
        else:
            image = '정보 없음'

        # 추출한 정보를 딕셔너리로 저장
        data = {
            "category": category,
            "title": title,
            "organizer": organizer,  # 수정된 부분
            "status": status_span,
            "period": period_td,
            "target": target,
            "prize": prize,
            "link": link,
            "image": image
        }

        return data
    else:
        print(f"페이지를 가져오는 데 실패했습니다. 상태 코드: {response.status_code}")
        return None


# 크롤링 결과를 JSON 파일로 저장하는 함수
def save_to_json(data, filename="incruit.json"):
    try:
        # 기존 데이터가 있으면 불러오기
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                all_data = json.load(f)
        except FileNotFoundError:
            all_data = []

        # 새 데이터를 추가
        all_data.append(data)

        # 파일에 저장
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(all_data, f, ensure_ascii=False, indent=4)
        print(f"데이터가 {filename} 파일에 저장되었습니다.")

    except Exception as e:
        print(f"파일 저장 중 오류가 발생했습니다: {e}")

# 크롤링 시작 함수
def crawl():
    start_contestno = 202411190006  # 시작 contestno
    end_contestno = 202411190000    # 끝 contestno (202409010101)

    # contestno가 감소하면서 크롤링 수행
    contestno = start_contestno
    while contestno >= end_contestno:
        print(f"\n--- 크롤링 중: {contestno} ---")
        
        # 페이지 정보 가져오기
        data = get_page_info(str(contestno))
        
        # 유효한 데이터가 있으면 JSON 파일에 저장
        if data:
            save_to_json(data)
        
        # contestno 업데이트 (하나씩 감소)
        contestno -= 1
        
        # 너무 빠르게 요청을 보내지 않도록 대기 시간 추가
        time.sleep(1)

# 크롤링 실행
crawl()



--- 크롤링 중: 202411190006 ---
데이터가 incruit.json 파일에 저장되었습니다.

--- 크롤링 중: 202411190005 ---
데이터가 incruit.json 파일에 저장되었습니다.

--- 크롤링 중: 202411190004 ---
데이터가 incruit.json 파일에 저장되었습니다.

--- 크롤링 중: 202411190003 ---
데이터가 incruit.json 파일에 저장되었습니다.

--- 크롤링 중: 202411190002 ---
데이터가 incruit.json 파일에 저장되었습니다.

--- 크롤링 중: 202411190001 ---
데이터가 incruit.json 파일에 저장되었습니다.

--- 크롤링 중: 202411190000 ---
데이터가 incruit.json 파일에 저장되었습니다.
